#### Create SparkSession

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("test_dataframe") \
            .getOrCreate()

### how to create spark dataframe:
  1. read
  ```python
    df1 = spark.read \
            .format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("sample.txt)
  ```
  2. sql()
  ```python
    df2 = spark.sql("""
    select count(distinct calltype) as distinct_call_type from fire_service_call_view where calltype is not NULL
    """)
  ```
  3. table(): convert spark table to spark dataframe
  ```python
    df3 = spark.table("spark_db_name.table_name")
  ```
  4. range(): create single column dataframe
  ```python
    df4 = spark.range(5)
  ```
  5. createDataFrame(): converts a list (python list, Spark Row, panda dataframe, RDD) into a spark dataframe
  ```python
    df1 = spark.createDataFrame(data_list)
  ```

In [0]:
df1 = spark.range(5)
df1.printSchema()
df1.show()

root
 |-- id: long (nullable = false)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [0]:
from datetime import datetime, date
data_list = [(1, 2., "string1", date(2022,1,1), datetime(2022,1,1,12,0)),
            (2, 3., "string2", date(2022,2,1), datetime(2022,1,2,12,0)),
            (3, 4., "string3", date(2022,3,1), datetime(2022,1,3,12,0))]

In [0]:
df1 = spark.createDataFrame(data_list).toDF("a","b","c","d","e")
df1.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2022-01-01|2022-01-01 12:00:00|
|  2|3.0|string2|2022-02-01|2022-01-02 12:00:00|
|  3|4.0|string3|2022-03-01|2022-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [0]:
df1.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
schema_1 = ["a","b","c","d","e"] 
schema_2 = "a int, b double, c string, d date, e timestamp"

In [0]:
spark.createDataFrame(data_list, schema=schema_1).printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
spark.createDataFrame(data_list, schema=schema_2).printSchema()

root
 |-- a: integer (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
from pyspark.sql import Row

row_list = [Row(a=1, b=2., c="string1", d=date(2022,1,1), e=datetime(2022,1,1,12,0)),
            Row(a=2, b=3., c="string2", d=date(2022,2,1), e=datetime(2022,1,2,12,0)),
            Row(a=3, b=4., c="string3", d=date(2022,3,1), e=datetime(2022,1,3,12,0))]

In [0]:
spark.createDataFrame(row_list).printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
import pandas as pd

pd_df = pd.DataFrame({"a": [1,2,3],
                     "b": [2.,3.,4.],
                     "c": ["string1", "string2", "string3"],
                     "d": [date(2022,1,1),date(2022,2,1),date(2022,3,1)],
                     "e": [datetime(2022,1,1,12,0),datetime(2022,1,2,12,0),datetime(2022,1,3,12,0)]})

In [0]:
spark.createDataFrame(pd_df, schema_2).printSchema()

root
 |-- a: integer (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
rdd = spark.sparkContext.parallelize([(1, 2., "string1", date(2022,1,1), datetime(2022,1,1,12,0)),
            (2, 3., "string2", date(2022,2,1), datetime(2022,1,2,12,0)),
            (3, 4., "string3", date(2022,3,1), datetime(2022,1,3,12,0))])

In [0]:
spark.createDataFrame(rdd, schema_1).printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [0]:
%fs ls /FileStore/tables


path,name,size,modificationTime
dbfs:/FileStore/tables/flight_time.json,flight_time.json,87150442,1654345450000


In [0]:
flight_schema_ddl = """FL_DATE date, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
             ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
             WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

1. Schema Inference is not advisable
1. often date, timestamp, boolean datatype can't store correct value into dataframe. Spark dataframe expect date in yyyy-mm-dd format normally. So if the data not loaded properly , it can store null without generating error. So add **mode** option.
2. **mode**: FAILFAST, DROPMALFORMED, PERMISSIVE
3. if there are 2 date columns, one is dd-mm-yyyy and another one is yyyy-mm-dd, then simply load the data as string datatype.
4. enforce schema explicitely.

In [0]:
flight_time_raw_df = spark.read \
                        .format("json") \
                        .schema(flight_schema_ddl) \
                        .option("mode", "FAILFAST") \
                        .option("dateFormat", "M/d/y") \
                        .load("/FileStore/tables/flight_time.json")

In [0]:
flight_time_raw_df.printSchema()

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DISTANCE: integer (nullable = true)



In [0]:
display(flight_time_raw_df)

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,CANCELLED,DISTANCE
2000-01-01,DL,1451,BOS,"Boston, MA",ATL,"Atlanta, GA",1115,1113,1343,5,1400,1348,0,946
2000-01-01,DL,1479,BOS,"Boston, MA",ATL,"Atlanta, GA",1315,1311,1536,7,1559,1543,0,946
2000-01-01,DL,1857,BOS,"Boston, MA",ATL,"Atlanta, GA",1415,1414,1642,9,1721,1651,0,946
2000-01-01,DL,1997,BOS,"Boston, MA",ATL,"Atlanta, GA",1715,1720,1955,10,2013,2005,0,946
2000-01-01,DL,2065,BOS,"Boston, MA",ATL,"Atlanta, GA",2015,2010,2230,10,2300,2240,0,946
2000-01-01,US,2619,BOS,"Boston, MA",ATL,"Atlanta, GA",650,649,956,7,955,1003,0,946
2000-01-01,US,2621,BOS,"Boston, MA",ATL,"Atlanta, GA",1440,1446,1713,4,1738,1717,0,946
2000-01-01,DL,346,BTR,"Baton Rouge, LA",ATL,"Atlanta, GA",1740,1744,1957,9,2008,2006,0,449
2000-01-01,DL,412,BTR,"Baton Rouge, LA",ATL,"Atlanta, GA",1345,1345,1552,9,1622,1601,0,449
2000-01-01,DL,299,BUF,"Buffalo, NY",ATL,"Atlanta, GA",1245,1245,1443,5,1455,1448,0,712


In [0]:
from pyspark.sql.types import *

flight_schema_struct = StructType([
    StructField("FL_DATE", DateType()),
    StructField("OP_CARRIER", StringType()),
    StructField("OP_CARRIER_FL_NUM", IntegerType()),
    StructField("ORIGIN", StringType()),
    StructField("ORIGIN_CITY_NAME", StringType()),
    StructField("DEST", StringType()),
    StructField("DEST_CITY_NAME", StringType()),
    StructField("CRS_DEP_TIME", StringType()),
    StructField("DEP_TIME", StringType()),
    StructField("WHEELS_ON", StringType()),
    StructField("TAXI_IN", StringType()),
    StructField("CRS_ARR_TIME", StringType()),
    StructField("ARR_TIME", StringType()),
    StructField("CANCELLED", StringType()),
    StructField("DISTANCE", StringType()),
])

In [0]:
flight_time_raw_df1 = spark.read \
                        .format("json") \
                        .schema(flight_schema_struct) \
                        .option("mode", "FAILFAST") \
                        .option("dateFormat", "M/d/y") \
                        .load("/FileStore/tables/flight_time.json")

In [0]:
display(flight_time_raw_df1)

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,CANCELLED,DISTANCE
2000-01-01,DL,1451,BOS,"Boston, MA",ATL,"Atlanta, GA",1115,1113,1343,5,1400,1348,0,946
2000-01-01,DL,1479,BOS,"Boston, MA",ATL,"Atlanta, GA",1315,1311,1536,7,1559,1543,0,946
2000-01-01,DL,1857,BOS,"Boston, MA",ATL,"Atlanta, GA",1415,1414,1642,9,1721,1651,0,946
2000-01-01,DL,1997,BOS,"Boston, MA",ATL,"Atlanta, GA",1715,1720,1955,10,2013,2005,0,946
2000-01-01,DL,2065,BOS,"Boston, MA",ATL,"Atlanta, GA",2015,2010,2230,10,2300,2240,0,946
2000-01-01,US,2619,BOS,"Boston, MA",ATL,"Atlanta, GA",650,649,956,7,955,1003,0,946
2000-01-01,US,2621,BOS,"Boston, MA",ATL,"Atlanta, GA",1440,1446,1713,4,1738,1717,0,946
2000-01-01,DL,346,BTR,"Baton Rouge, LA",ATL,"Atlanta, GA",1740,1744,1957,9,2008,2006,0,449
2000-01-01,DL,412,BTR,"Baton Rouge, LA",ATL,"Atlanta, GA",1345,1345,1552,9,1622,1601,0,449
2000-01-01,DL,299,BUF,"Buffalo, NY",ATL,"Atlanta, GA",1245,1245,1443,5,1455,1448,0,712


#### There are 2 approaches to write the dataframe
1. read -> transform -> create view using dataframe -> create spark table -> Load into the table
2. read -> transform -> save as a spark table

In [0]:
from pyspark.sql.functions import to_date, expr

flight_time_df = flight_time_raw_df \
                    .withColumn("FL_DATE", to_date("FL_DATE", "M/d/y")) \
                    .withColumn("CANCELLED", expr("if(CANCELLED==1, true, false)"))

#### convert spark dataframe into a spark table 
1. dataframe write mode can be
    1. append
    2. overwrite
    3. error **check
    4.

In [0]:
%fs rm -r /user/hive/warehouse/flight_time_tbl

In [0]:
flight_time_df.write \
            .format("parquet") \
            .mode("overwrite") \
            .saveAsTable("flight_time_tbl")

In [0]:
%sql
describe extended flight_time_tbl

col_name,data_type,comment
FL_DATE,date,null
OP_CARRIER,string,null
OP_CARRIER_FL_NUM,int,null
ORIGIN,string,null
ORIGIN_CITY_NAME,string,null
DEST,string,null
DEST_CITY_NAME,string,null
CRS_DEP_TIME,int,null
DEP_TIME,int,null
WHEELS_ON,int,null


In [0]:
%fs ls /user/hive/warehouse/flight_time_tbl

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_time_tbl/_SUCCESS,_SUCCESS,0,1654350154000
dbfs:/user/hive/warehouse/flight_time_tbl/_committed_8038701575572934755,_committed_8038701575572934755,816,1654350154000
dbfs:/user/hive/warehouse/flight_time_tbl/_started_8038701575572934755,_started_8038701575572934755,0,1654350140000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00000-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-43-1-c000.snappy.parquet,part-00000-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-43-1-c000.snappy.parquet,501013,1654350153000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00001-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-44-1-c000.snappy.parquet,part-00001-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-44-1-c000.snappy.parquet,492268,1654350153000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00002-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-45-1-c000.snappy.parquet,part-00002-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-45-1-c000.snappy.parquet,485042,1654350153000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00003-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-46-1-c000.snappy.parquet,part-00003-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-46-1-c000.snappy.parquet,488176,1654350152000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00004-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-47-1-c000.snappy.parquet,part-00004-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-47-1-c000.snappy.parquet,481072,1654350153000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00005-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-48-1-c000.snappy.parquet,part-00005-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-48-1-c000.snappy.parquet,498348,1654350152000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00006-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-49-1-c000.snappy.parquet,part-00006-tid-8038701575572934755-d5ecd922-bd6a-483a-950b-87369a36bfed-49-1-c000.snappy.parquet,497461,1654350153000


#### How to check table metadata

In [0]:
%sql
show tables

database,tableName,isTemporary
default,flight_time_tbl,false


In [0]:
spark.catalog.listTables()

Out[48]: [Table(name='flight_time_tbl', database='default', description=None, tableType='MANAGED', isTemporary=False)]

#### Managed vs External Tables
1. saveAsTable abd Create table DDl by command both create Managed Table default
2. **for Managed Table**, spark creates table at a pre-defined warehouse location. You can specify the location befor creating the cluster by specifying the "spark.sql.warehouse.dir" config. But you can't change once defined.
3. Spark manages table metadata and table data together.
4. spark deletes the table data and metadata both when you drop the table.


1. **External Table** is a mechanism to share the data across the projects or different storage layers.
2. using location in create table ddl command to create external table.
3. You can't drop the external table (drop only metadata not the actual data), but can read, update or overwrite it like managed table.

In [0]:
%sql

create table ext_flight_time_tbl like flight_time_tbl
location "/user/hive/warehouse/flight_time_tbl"

In [0]:
%sql
describe extended ext_flight_time_tbl

col_name,data_type,comment
FL_DATE,date,null
OP_CARRIER,string,null
OP_CARRIER_FL_NUM,int,null
ORIGIN,string,null
ORIGIN_CITY_NAME,string,null
DEST,string,null
DEST_CITY_NAME,string,null
CRS_DEP_TIME,int,null
DEP_TIME,int,null
WHEELS_ON,int,null


In [0]:
spark.catalog.listTables()

Out[49]: [Table(name='ext_flight_time_tbl', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='flight_time_tbl', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
%sql
drop table ext_flight_time_tbl